<a href="https://colab.research.google.com/github/anirudh1906/Major_Project/blob/main/Vader_Sentiment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs =SentimentIntensityAnalyzer()

In [ ]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import os

In [ ]:
url= "http://www.inshorts.com/en/read/sports"
news_data = []
news_category=url.split('/')[-1]
news_category
data=requests.get(url)
soup=BeautifulSoup(data.content)
#print(soup)

In [ ]:
urls= ['http://www.inshorts.com/en/read/sports',
       'http://www.inshorts.com/en/read/world',
       'http://www.inshorts.com/en/read/politics']

def build_dataset(urls):  
  news_data = []
  for url in urls:
    news_category=url.split('/')[-1]
    data=requests.get(url)
    soup=BeautifulSoup(data.content)

    news_articles=[{"news_headline":headline.find('span',attrs={"itemprop":"headline"}).string,
                  "news_article":article.find('div',attrs={"itemprop":"articleBody"}).string,
                  "news_category":news_category}
                  
                  for headline,article in zip(soup.find_all("div",class_=["news-card-title news-right-box"]),
                                              soup.find_all("div",class_=["news-card-content news-right-box"]))]
  
    news_articles=news_articles[0:20]
    news_data.extend(news_articles)
  
  
  df=pd.DataFrame(news_data)
  df=df[["news_headline","news_article","news_category"]]
  return df

  

In [ ]:
df= build_dataset(urls)
df.head()

,news_headline,news_article,news_category
0,"I was abused for saying it's raining, not waki...",Cricketer-commentator Dinesh Karthik has revea...,sports
1,Bangladesh's Taskin does dance step while batt...,Bangladesh's Taskin Ahmed did a dance step aft...,sports
2,"Move over Gal Gadot, the real Wonder Woman is ...",India Women's Harleen Deol pulled off a catch ...,sports
3,Djokovic defeats Shapovalov in Wimbledon semi-...,World number one Novak Djokovic defeated world...,sports
4,Pakistan were pathetic: Vaughan after England ...,After England defeated Pakistan by 9 wickets i...,sports


In [ ]:
import nltk
nltk.download("stopwords")
stopwords_list=nltk.corpus.stopwords.words("english")
stopwords_list.remove("no")
stopwords_list.remove("not")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

In [ ]:
!pip install contractions
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand


In [ ]:
import re
def remove_sp(text):
   pattern=r"[^a-zA-Z0-9\s]"
   text=re.sub(pattern,'',text)
   return text

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopwords_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()
# lemmatize string
def lemmatize_word(text):
    word_tokens = word_tokenize(text)
    # provide context i.e. part-of-speech
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    return lemmas

In [ ]:
df.news_headline=df.news_headline.apply(lambda x:x.lower())
df.news_article=df.news_article.apply(lambda x:x.lower())

df.news_headline=df.news_headline.apply(html_tag)
df.news_article=df.news_article.apply(html_tag)

df.news_headline=df.news_headline.apply(con)
df.news_article=df.news_article.apply(con)

df.news_headline=df.news_headline.apply(remove_sp)
df.news_article=df.news_article.apply(remove_sp)

df.news_headline=df.news_headline.apply(remove_stopwords)
df.news_article=df.news_article.apply(remove_stopwords)

df.news_headline=df.news_headline.apply(lemmatize_word)
df.news_article=df.news_article.apply(lemmatize_word)


In [ ]:
df["compound"]=df["news_article"].apply(lambda x: vs.polarity_scores(x)["compound"])

In [ ]:
df.shape

(60, 4)

In [ ]:
for i in range(0,60):
  if df["compound"][i]>=0.05:
    df["compound"][i]="positive"
  elif df["compound"][i]>-0.05 and df["compound"][i]<0.05:
    df["compound"][i]="neutral"
  elif df["compound"][i]<=-0.05:
    df["compound"][i]="negative"


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
import joblib
joblib.dump(SentimentAnalysis)

NameError: ignored